In [4]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

In [8]:
url = "https://www.thebootranchapartments.com/floorplans"

In [9]:
# Send a GET request to fetch the webpage content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [10]:
def extract_rent_info(soup):
    # List to hold the data
    rent_data = []
    
    # Example: assuming each floor plan is inside a container (adjust based on actual HTML structure)
    floorplans = soup.find_all('div', class_='floorplanInfo')  # You may need to adjust the class name here
    
    if not floorplans:
        print("No floorplans found!")
        return rent_data

    for plan in floorplans:
        try:
            availability = plan.find('div', class_='floorplanAvailability')
            if availability:
                availability = availability.get_text(strip=True)
            else:
                availability = "N/A"
            
            # Extracting the name of the floor plan
            name = plan.find('div', class_='floorplanName')
            if name:
                name = name.get_text(strip=True)
            else:
                name = "N/A"
            
            # Extracting stats: beds, baths, sqft
            beds = plan.find('div', class_='floorplanBeds')
            if beds:
                beds = beds.find('div', class_='statValue').get_text(strip=True)
            else:
                beds = "N/A"
            
            baths = plan.find('div', class_='floorplanBaths')
            if baths:
                baths = baths.find('div', class_='statValue').get_text(strip=True)
            else:
                baths = "N/A"
            
            sqft = plan.find('div', class_='floorplanSquareFootage')
            if sqft:
                sqft = sqft.find('div', class_='statValue').get_text(strip=True)
            else:
                sqft = "N/A"

            # Extracting the price
            price = plan.find('span', class_='priceValueOverride')
            if price:
                price = price.get_text(strip=True)
            else:
                price = "N/A"

            # Append the data to list
            rent_data.append([availability, name, beds, baths, sqft, price])

        except AttributeError as e:
            print(f"Error: {e}")  # Print the error if something goes wrong

    return rent_data

In [11]:
rent_info = extract_rent_info(soup)

In [12]:
for row in rent_info:
    print(row)

['Only 1 Vacant Unit Left!', 'D1', '4', '2', '1456', '$2424']
['Available', 'C1', '3', '2', '1327', '$2170']
['Available on October 4, 2024', 'B1', '2', '1', '967', '$2121']
['Only 2 Vacant Units Left!', 'B2', '2', '2', '1128', '$2106']
['Only 1 Vacant Unit Left!', 'A3', '1', '1', '790', '$1598']
['Only 2 Vacant Units Left!', 'A2', '1', '1', '784', '$1443']
['Only 2 Vacant Units Left!', 'A1', '1', '1', '633', '$1403']


In [20]:
apartment_name = "The Boot Ranch Apartments"

# Update each row to include the apartment name
for row in rent_info:
    row.insert(0, apartment_name) 

for row in rent_info:
    print(row)

['The Boot Ranch Apartments', 'Only 1 Vacant Unit Left!', 'D1', '4', '2', '1456', '$2424']
['The Boot Ranch Apartments', 'Available', 'C1', '3', '2', '1327', '$2170']
['The Boot Ranch Apartments', 'Available on October 4, 2024', 'B1', '2', '1', '967', '$2121']
['The Boot Ranch Apartments', 'Only 2 Vacant Units Left!', 'B2', '2', '2', '1128', '$2106']
['The Boot Ranch Apartments', 'Only 1 Vacant Unit Left!', 'A3', '1', '1', '790', '$1598']
['The Boot Ranch Apartments', 'Only 2 Vacant Units Left!', 'A2', '1', '1', '784', '$1443']
['The Boot Ranch Apartments', 'Only 2 Vacant Units Left!', 'A1', '1', '1', '633', '$1403']


In [21]:
def add_price_per_sqft(data):
    for row in data:
        try:
            # Extract price (remove dollar sign and commas) and convert to float
            price = float(row[6].replace('$', '').replace(',', ''))
            
            # Extract square footage and convert to float
            sqft = float(row[5])
            
            # Calculate price per square foot
            price_per_sqft = price / sqft
            
            # Append the result (formatted to 2 decimal places)
            row.append(f"{price_per_sqft:.2f}")
        except (ValueError, IndexError) as e:
            # Handle any conversion errors
            print(f"Error processing row {row}: {e}")
            row.append('N/A')  # Append 'N/A' if there's an issue

# Add the calculation to each row
add_price_per_sqft(rent_info)

for row in rent_info:
    print(row)

['The Boot Ranch Apartments', 'Only 1 Vacant Unit Left!', 'D1', '4', '2', '1456', '$2424', '1.66']
['The Boot Ranch Apartments', 'Available', 'C1', '3', '2', '1327', '$2170', '1.64']
['The Boot Ranch Apartments', 'Available on October 4, 2024', 'B1', '2', '1', '967', '$2121', '2.19']
['The Boot Ranch Apartments', 'Only 2 Vacant Units Left!', 'B2', '2', '2', '1128', '$2106', '1.87']
['The Boot Ranch Apartments', 'Only 1 Vacant Unit Left!', 'A3', '1', '1', '790', '$1598', '2.02']
['The Boot Ranch Apartments', 'Only 2 Vacant Units Left!', 'A2', '1', '1', '784', '$1443', '1.84']
['The Boot Ranch Apartments', 'Only 2 Vacant Units Left!', 'A1', '1', '1', '633', '$1403', '2.22']


In [15]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f'C:/Users/onlyu/rent_data_{timestamp}.csv'
def save_to_csv(data, filename=filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(['Availability', 'Name', 'Beds', 'Baths', 'Square Feet', 'Price'])
        # Write the data rows
        writer.writerows(data)

In [16]:
save_to_csv(rent_info)

In [17]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import os

In [24]:
sender_email = 'gregoryfulham@gmail.com'
receiver_email = 'g_fulham@yahoo.com'
subject = 'Daily Rent Data'
body = 'Please find attached the rent data CSV file.'
password = 'ywqs bjcg mcmz vlxn'

# Set up the MIME
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject

# Create the HTML table for the email body
html = """\
<html>
  <body>
    <p>Hello,<br><br>
       Please find the latest rent data below:
    </p>
    <table border="1" cellpadding="5" cellspacing="0">
      <tr>
        <th>Apartment Name</th>
        <th>Availability</th>
        <th>Floorplan Name</th>
        <th>Beds</th>
        <th>Baths</th>
        <th>Square Feet</th>
        <th>Price</th>
        <th>Price per Sq Ft</th>
      </tr>
"""

# Add rows to the HTML table
for row in rent_info:
    html += f"<tr><td>{row[0]}</td><td>{row[1]}</td><td>{row[2]}</td><td>{row[3]}</td><td>{row[4]}</td><td>{row[5]}</td><td>{row[6]}</td><td>{row[7]}</td></tr>"

# Close the table and HTML structure
html += """\
    </table>
    <br>
    Regards,<br>
    Your Name
  </body>
</html>
"""

# Set up the MIME
msg = MIMEMultipart("alternative")
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject

# Attach the HTML content to the email
msg.attach(MIMEText(html, 'html'))

# SMTP server setup (Gmail example)
smtp_server = 'smtp.gmail.com'
port = 587  # For TLS

# Send the email
try:
    server = smtplib.SMTP(smtp_server, port)
    server.starttls()  # Secure the connection
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    print('Email sent successfully!')
except Exception as e:
    print(f'Error: {e}')
finally:
    server.quit()


Email sent successfully!
